# Simple Go-To-Goal for Cerus

The following code implements a simple go-to-goal behavior for Cerus. It uses a closed feedback loop to continuously asses Cerus' state (position and heading) in the world using data from two wheel encoders. It subsequently calculates the error between a given goal location and its current pose and will attempt to minimize the error until it reaches the goal location. A P-regulator (see PID regulator) script uses the error as an input and outputs the angular velocity for the Arduino and motor controllers that drive the robot. 

All models used in this program are adapted from Georgia Tech's "Control of Mobile Robots" by Dr. Magnus Egerstedt.    

In [ ]:
#Import useful libraries
import serial
import time
import math
import numpy as np
from traitlets import HasTraits, List

In [ ]:
#Open a serial connection with the Arduino Mega
#Opening a serial port on the Arduino resets it, so our encoder count is also reset to 0,0
ser = serial.Serial('COM3', 115200)

In [ ]:
#Defining our goal location. Units are metric, real-world coordinates in an X/Y coordinate system
goal_x = 1
goal_y = 0

In [ ]:
#Create a class for our Cerus robot
class Cerus():
    def __init__(self, pose_x, pose_y, pose_phi, R_wheel, N_ticks, L_track):
        self.pose_x = pose_x #X Position
        self.pose_y = pose_y #Y Position
        self.pose_phi = pose_phi #Heading
        self.R_wheel = R_wheel #wheel radius in meters
        self.N_ticks = N_ticks #encoder ticks per wheel revolution
        self.L_track = L_track #wheel track in meters

#Create a Cerus instance and initialize it to a 0,0,0 world position and with some physical dimensions 
cerus = Cerus(0,0,0,0.03,900,0.23)

We'll use the Traitlets library to implement an observer pattern that will recalculate the pose of the robot every time an update to the encoder values is detected and sent to the Jetson nano by the Arduino.

In [ ]:
#Create an encoder class with traits
class Encoders(HasTraits):    
    encoderValues = List() #We store the left and right encoder value in a list
    
    def __init__(self, encoderValues, deltaTicks):
        self.encoderValues = encoderValues        
        self.deltaTicks = deltaTicks       

#Create an encoder instance 
encoders = Encoders([0,0], [0,0])

In [ ]:
#Create a function that is triggered when a change to encoders is detected
def monitorEncoders(change):
        if change['new']:
                       
            oldVals = np.array(change['old'])
            newVals = np.array(change['new'])
            deltaTicks = newVals - oldVals
            #print("Old values: ", oldVals)
            #print("New values: ", newVals)
            print("Delta values: ", deltaTicks)
            
            calculatePose(deltaTicks)
            
encoders.observe(monitorEncoders, names = "encoderValues")

The functions below are helpers and will be called through our main loop.

In [ ]:
#Create a move function that sends move commands to the Arduino
def move(linearVelocity, angularVelocity):       
        
        command = f"<{linearVelocity},{angularVelocity}>"
        ser.write(str.encode(command))

In [ ]:
#Create a function that calculates an updated pose of Cerus every time it is called
def calculatePose(deltaTicks):
    
    #Calculate the centerline distance moved
    distanceLeft = 2 * math.pi * cerus.R_wheel * (deltaTicks[0] / cerus.N_ticks)
    distanceRight = 2 * math.pi * cerus.R_wheel * (deltaTicks[1] / cerus.N_ticks)
    distanceCenter = (distanceLeft + distanceRight) / 2
        
    #Update the position and heading
    cerus.pose_x = round((cerus.pose_x + distanceCenter * math.cos(cerus.pose_phi)), 4)
    cerus.pose_y = round((cerus.pose_y + distanceCenter * math.sin(cerus.pose_phi)), 4)
    cerus.pose_phi = round((cerus.pose_phi + ((distanceRight - distanceLeft) / cerus.L_track)), 4)
        
    print(f"The new position is {cerus.pose_x}, {cerus.pose_y} and the new heading is {cerus.pose_phi}.")

In [ ]:
#Calculate the error between Cerus' heading and the goal point
def calculateError():
    
    phi_desired = math.atan((goal_y - cerus.pose_y)/(goal_x - cerus.pose_x))
    
    temp = phi_desired - cerus.pose_phi
    error_heading = round((math.atan2(math.sin(temp), math.cos(temp))), 4) #ensure that error is within [-pi, pi]
    error_x = round((goal_x - cerus.pose_x), 4)
    error_y = round((goal_y - cerus.pose_y), 4)
    
    #print("The heading error is: ", error_heading)
    #print("The X error is: ", error_x)
    #print("The Y error is: ", error_y)
    return error_x, error_y, error_heading

In [ ]:
#Functions to read and format encoder data received from the Serial port
def formatData(data):
    delimiter = "x"
    leftVal = ""
    rightVal = ""
    
    for i in range(len(data)):        
        if data[i] == ",":
            delimiter = ","            
        elif delimiter != "," and data[i].isdigit():
            leftVal += data[i]            
        elif delimiter == "," and data[i].isdigit():
            rightVal += data[i]    
       
    leftVal, rightVal = int(leftVal), int(rightVal)
    encoders.encoderValues = [leftVal, rightVal]    
    print("Encoders: ", encoders.encoderValues)

def handleSerial():
    #ser.readline() waits for the next line of encoder data, which is sent by Arduino every 50 ms
    temp = ser.readline()
    data = temp.decode()
    formatData(data)    

This is the main part for our program that will loop over and over until Cerus has reached its goal. For our simple go-to-goal behavior, we will drive the robot at a constant speed and only adjust our heading so that we reach the goal location.

__WARNING: This will move the robot!__

In [ ]:
atGoal = False
constVel = 0.2

while not atGoal:
    try:        
        
        #Calculate the current pose to goal 
        error_x, error_y, error_heading = calculateError()
        
        #If we're within 5 cm of the goal
        if error_x <= 0.05: #and error_y <= 0.05:
            print("Goal reached!")
            move(0.0,0.0)
            time.sleep(0.1)
            atGoal = True

        #Otherwise keep driving
        else:
            K = 1 #constant for our P-regulator below
            omega = K * error_heading            
            handleSerial()  
            #ADD ANGULAR VELOCITY HERE
            move(constVel,0.0)
            print("Moving")
            
                                    
    except(KeyboardInterrupt):
        print("Program interrupted by user!")
        move(0.0,0.0) #Stop motors
        break

"Loop exited..."
move(0.0,0.0) #Stop motors

In [ ]:
#Close the serial connection when done
ser.close()